<a href="https://colab.research.google.com/github/Flammingo2/TensorFlow/blob/main/homework3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [113]:
import numpy as np
%tensorflow_version 2.x
import tensorflow as tf
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds

In [118]:
# Load the dataset in Tupel-Format (input, label)
train_dataset, test_dataset = tfds.load('genomics_ood', split = ['train[:1%]', 'test[:1%]'], as_supervised=True)

train_seq, train_labels = zip(*train_dataset)
test_seq, test_labels = zip(*test_dataset)

print(train_seq[0])
print(train_labels[0])
print(train_seq[1])
print(train_labels[1])

tf.Tensor(b'AAACTATGTTATATTCACGATGATTAACTTACAAAGGAGTTTCAACTATGAAGATGATAAACAAATTAATCGTTCCGGTAACAGCTAGTGCTTTATTATTAGGCGCTTGTGGCGCTAGTGCCACAGACTCTAAAGAAAATACATTAATTTCTTCTAAAGCTGGAGACGTAACAGTTGCAGATACAATGAAAAAAATCGGTAAAGATCAAATTGCAAATGCATCATTTACTGAAATGTTAAATAAAATTTT', shape=(), dtype=string)
tf.Tensor(7, shape=(), dtype=int64)
tf.Tensor(b'CACAGCCGGCCGCTGACCTGCTGGCCATCGGAGCGCTGGCCGGCCTCGAGGATATTGGCCAGCAGCAGGTGGATGTCTCGGGCATAGCGCTCCCCCTGGTAGGTGATGCGAATGCTGCGGCCCTGGCGCTCGGTCAGGGCAAAACCCAGGGTCTGCTCCAGGCTCTTGATCTGGTGGCTGATGGCGCTGGGCGTCAGGTTCAGCTCATTGGCGGCCTCGGCGACACTGCCCAGGCGGGCCACCGCGTCCA', shape=(), dtype=string)
tf.Tensor(5, shape=(), dtype=int64)


In [119]:
#Look at shapes
print(len(train_seq))
print(len(train_labels))
print(len(test_seq))
print(len(test_labels))

print(train_labels[0])
print(train_seq[0])
print(type(train_labels))

10000
10000
1000
1000
tf.Tensor(7, shape=(), dtype=int64)
tf.Tensor(b'AAACTATGTTATATTCACGATGATTAACTTACAAAGGAGTTTCAACTATGAAGATGATAAACAAATTAATCGTTCCGGTAACAGCTAGTGCTTTATTATTAGGCGCTTGTGGCGCTAGTGCCACAGACTCTAAAGAAAATACATTAATTTCTTCTAAAGCTGGAGACGTAACAGTTGCAGATACAATGAAAAAAATCGGTAAAGATCAAATTGCAAATGCATCATTTACTGAAATGTTAAATAAAATTTT', shape=(), dtype=string)
<class 'tuple'>


In [120]:
def onehotify(tensor):
  vocab = {'A':'1', 'C': '2', 'G':'3', 'T':'0'}
  for key in vocab.keys():
    tensor = tf.strings.regex_replace(tensor, key, vocab[key])
  split = tf.strings.bytes_split(tensor)
  labels = tf.cast(tf.strings.to_number(split), tf.uint8)
  onehot = tf.one_hot(labels, 4)
  onehot = tf.reshape(onehot, (-1,))
  return onehot

In [121]:
#Create tensors from lists
# 1. for training dataset
#for sample in train_seq:
#  train_seq = onehotify(sample)
#train_set_seq = tf.data.Dataset.from_tensor_slices(train_seq)

for sample in train_labels:
  print(sample.shape)
  tf.make_ndarray(sample)
  print(sample)
  train_labels = sample.map(lambda t : tf.one_hot(t, 10, on_value=1.0, off_value=0.0))
print(type(train_labels))
print(train_labels[0])
train_set_labels = tf.data.Dataset.from_tensor_slices(train_labels)

# 2. for test dataset
for sample in test_seq:
  test_seq = onehotify(sample)
test_set_seq = tf.data.Dataset.from_tensor_slices(test_seq)

test_labels = test_labels.map(lambda t : tf.one_hot(t,10))
test_set_labels = tf.data.Dataset.from_tensor_slices(test_labels)

()


AttributeError: ignored

In [ ]:
#Reshape to 1D tensors
# 1. train dataset
train_set_seq = train_seq.map(lambda seq : tf.reshape(seq, (-1,)))
train_set_labels = train_set_labels.map(lambda t : tf.one_hot(t, 10))
# 2. test dataset
test_set_seq = test_set_seq.map(lambda img : tf.reshape(img, (-1,)))
test_set_labels = test_set_labels.map(lambda t : tf.one_hot(t,10))

#Zip together input images and targets
# 1. for training dataset
train_dataset = tf.data.Dataset.zip((train_set_seq, train_set_labels)).batch(128).shuffle(buffer_size=128)
# 2. for test dataset
test_dataset = tf.data.Dataset.zip((test_set_seq, test_set_labels)).batch(128).shuffle(buffer_size=128)